# Chapter 3

### Our first TensorFlow graph

In [11]:
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()

with tf.compat.v1.Session() as sess: 
    a = tf.constant(5) 
    b = tf.constant(2)
    c = tf.constant(3)
    d = tf.multiply(a,b) 
    e = tf.add(c,b) 
    f = tf.subtract(d,e) 

    outs = sess.run(f) 
    sess.close() 
    print("outs = {}".format(outs))

outs = 5


### Constructing and managing our graph 

In [12]:
import tensorflow as tf
print(tf.compat.v1.get_default_graph())
g = tf.Graph()
print(g)

In [13]:
a = tf.compat.v1.constant(5) 
print(a.graph is g)
print(a.graph is tf.compat.v1.get_default_graph())

False
True


In [15]:
g1 = tf.compat.v1.get_default_graph() 
g2 = tf.Graph() 

print(g1 is tf.compat.v1.get_default_graph())

with g2.as_default(): 
    print(g1 is tf.compat.v1.get_default_graph())

print(g1 is tf.compat.v1.get_default_graph())

True
False
True


### Fetches 

In [17]:
with tf.compat.v1.Session() as sess: 
    fetches = [a,b,c,d,e,f]
    outs = sess.run(fetches) 

print("outs = {}".format(outs))
print(type(outs[0]))

outs = [5, 2, 3, 10, 5, 5]
<class 'numpy.int32'>


### Nodes are operations, edges are Tensor objects  

In [18]:
c = tf.constant(4.0)
print(c)

Tensor("Const_12:0", shape=(), dtype=float32)


### Data types  

In [19]:
c = tf.constant(4.0, dtype=tf.float64)
print(c)
print(c.dtype)

Tensor("Const_13:0", shape=(), dtype=float64)
<dtype: 'float64'>


In [20]:
x = tf.constant([1,2,3],name='x',dtype=tf.float32) 
print(x.dtype)
x = tf.cast(x,tf.int64)
print(x.dtype)

<dtype: 'float32'>
<dtype: 'int64'>


### Tensor arrays and Shapes  

In [21]:
import numpy as np 

c = tf.constant([[1,2,3],
                 [4,5,6]]) 
print("Python List input: {}".format(c.get_shape()))

c = tf.constant(np.array([
                 [[1,2,3], 
                  [4,5,6]], 

                 [[1,1,1], 
                  [2,2,2]]
                 ])) 

print("3d Numpy array input: {}".format(c.get_shape()))

Python List input: (2, 3)
3d Numpy array input: (2, 2, 3)


In [22]:
import matplotlib.pyplot as plt 
% matplotlib inline

sess = tf.InteractiveSession()

# === Noramal and Truncated normal distributions ===
mean = 0
std = 1
x_normal = tf.random_normal((1,50000),mean,std).eval()
x_truncated = tf.truncated_normal((1,50000),mean,std).eval()

# === Uniform distribution
minval = -2 
maxval = 2
x_uniform = tf.random_uniform((1,50000),minval,maxval).eval()

sess.close()


def simpleaxis(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
#     ax.set_ylim([-1.1,1.1])
    ax.tick_params(axis='both', which='major', labelsize=15)
    
def get_axis_limits(ax, scale=.8):
    return ax.get_xlim()[1]*scale, ax.get_ylim()[1]*scale

f,axarr = plt.subplots(1,3,figsize=[15,4],sharey=True)
titles = ['Normal','Truncated Normal','Uniform']

print(x_normal.shape)
for i,x in enumerate([x_normal,x_truncated,x_uniform]):
    ax = axarr[i]
    ax.hist(x[0],bins=100,color='b',alpha=0.4)
    ax.set_title(titles[i],fontsize=20)
    ax.set_xlabel('Values',fontsize=20)
    ax.set_xlim([-5,5])
    ax.set_ylim([0,1800])
    
    simpleaxis(ax)
    
    
axarr[0].set_ylabel('Frequency',fontsize=20)
plt.suptitle('Initialized values',fontsize=30, y=1.15)


for ax,letter in zip(axarr,['A','B','C']):
    simpleaxis(ax)
    ax.annotate(letter, xy=get_axis_limits(ax),fontsize=35)

plt.tight_layout()

plt.savefig('histograms.png', bbox_inches='tight', format='png', dpi=200, pad_inches=0,transparent=True)

plt.show()

UsageError: Line magic function `%` not found.


In [23]:
sess = tf.InteractiveSession()
c = tf.linspace(0.0, 4.0, 5) 
print("The content of 'c':\n {}\n".format(c.eval()))
sess.close()

AttributeError: module 'tensorflow' has no attribute 'InteractiveSession'

### Matrix multiplication

In [5]:
A = tf.constant([ [1,2,3],
                  [4,5,6] ])
print(a.get_shape())

x = tf.constant([1,0,1])
print(x.get_shape())

x = tf.expand_dims(x,1)
print(x.get_shape())

b = tf.matmul(A,x)

sess = tf.InteractiveSession()
print('matmul result:\n {}'.format(b.eval()))
sess.close()

()
(3,)
(3, 1)
matmul result:
 [[ 4]
 [10]]


### Names  

In [18]:
with tf.Graph().as_default():
    c1 = tf.constant(4,dtype=tf.float64,name='c') 
    c2 = tf.constant(4,dtype=tf.int32,name='c') 
print(c1.name)
print(c2.name)

c:0
c_1:0


### Name scopes

In [19]:
with tf.Graph().as_default():
    c1 = tf.constant(4,dtype=tf.float64,name='c') 
    with tf.name_scope("prefix_name"):
        c2 = tf.constant(4,dtype=tf.int32,name='c') 
        c3 = tf.constant(4,dtype=tf.float64,name='c')

print(c1.name)
print(c2.name)
print(c3.name)

c:0
prefix_name/c:0
prefix_name/c_1:0


### Variables


In [20]:
init_val = tf.random_normal((1,5),0,1)
var = tf.Variable(init_val, name='var') 
print("pre run: \n{}".format(var))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)

print("\npost run: \n{}".format(post_var))

pre run: 
<tf.Variable 'var:0' shape=(1, 5) dtype=float32_ref>

post run: 
[[-0.69993478  1.55560553  0.55664998 -1.7586478   3.34199762]]


### New variables are created each time

In [21]:
init_val = tf.random_normal((1,5),0,1)
var = tf.Variable(init_val, name='var') 
print("pre run: \n{}".format(var))

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    post_var = sess.run(var)

print("\npost run: \n{}".format(post_var))

pre run: 
<tf.Variable 'var_1:0' shape=(1, 5) dtype=float32_ref>

post run: 
[[-0.97382897 -0.38875952 -1.22835159 -0.75114495  0.50170541]]


### Placeholders

In [22]:
x_data = np.random.randn(5,10) 
w_data = np.random.randn(10,1) 

with tf.Graph().as_default():
    x = tf.placeholder(tf.float32,shape=(5,10))
    w = tf.placeholder(tf.float32,shape=(10,1))
    b = tf.fill((5,1),-1.) 
    xw = tf.matmul(x,w)

    xwb = xw + b
    s = tf.reduce_max(xwb)
    with tf.Session() as sess:
        outs = sess.run(s,feed_dict={x: x_data,w: w_data}) 

print("outs = {}".format(outs))

outs = 6.63996839523


### Example 1: Linear Regression

In [23]:
# === Create data and simulate results =====
x_data = np.random.randn(2000,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2

noise = np.random.randn(1,2000)*0.1
y_data = np.matmul(w_real,x_data.T) + b_real + noise

In [24]:
NUM_STEPS = 10

g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x)) + b

    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(tf.square(y_true-y_pred))
  
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)

    # Before starting, initialize the variables.  We will 'run' this first.
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)      
        for step in range(NUM_STEPS):
            sess.run(train,{x: x_data, y_true: y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b])) 
                wb_.append(sess.run([w,b]))
                
        print(10, sess.run([w,b]))

(0, [array([[ 0.30298612,  0.48779276,  0.0716765 ]], dtype=float32), -0.18869072])
(5, [array([[ 0.3005667 ,  0.50066561,  0.09561971]], dtype=float32), -0.19749904])
(10, [array([[ 0.3005667 ,  0.50066561,  0.0956197 ]], dtype=float32), -0.19749907])


### Example 2: Logistic Regression


In [25]:
N = 20000

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# === Create data and simulate results =====
x_data = np.random.randn(N,3)
w_real = [0.3,0.5,0.1]
b_real = -0.2
wxb = np.matmul(w_real,x_data.T) + b_real

y_data_pre_noise = sigmoid(wxb)
y_data = np.random.binomial(1,y_data_pre_noise)

In [26]:
NUM_STEPS = 50


g = tf.Graph()
wb_ = []
with g.as_default():
    x = tf.placeholder(tf.float32,shape=[None,3])
    y_true = tf.placeholder(tf.float32,shape=None)
    
    with tf.name_scope('inference') as scope:
        w = tf.Variable([[0,0,0]],dtype=tf.float32,name='weights')
        b = tf.Variable(0,dtype=tf.float32,name='bias')
        y_pred = tf.matmul(w,tf.transpose(x)) + b

    with tf.name_scope('loss') as scope:
        loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_true,logits=y_pred) 
        loss = tf.reduce_mean(loss)
  
    with tf.name_scope('train') as scope:
        learning_rate = 0.5
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        train = optimizer.minimize(loss)



    # Before starting, initialize the variables.  We will 'run' this first.
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)      
        for step in range(NUM_STEPS):
            sess.run(train,{x: x_data, y_true: y_data})
            if (step % 5 == 0):
                print(step, sess.run([w,b]))
                wb_.append(sess.run([w,b]))

        print(50, sess.run([w,b]))

(0, [array([[ 0.03320302,  0.05792972,  0.01119308]], dtype=float32), -0.020700064])
(5, [array([[ 0.14649118,  0.2548342 ,  0.04982107]], dtype=float32), -0.091209568])
(10, [array([[ 0.20652841,  0.35850295,  0.07069619]], dtype=float32), -0.12844093])
(15, [array([[ 0.23972291,  0.41549107,  0.08243967]], dtype=float32), -0.14894117])
(20, [array([[ 0.25859103,  0.44772011,  0.08921722]], dtype=float32), -0.16054493])
(25, [array([[ 0.26949188,  0.4662571 ,  0.09318593]], dtype=float32), -0.16722204])
(30, [array([[ 0.27584928,  0.47702512,  0.09552839]], dtype=float32), -0.17110163])
(35, [array([[ 0.27957708,  0.48331678,  0.09691675]], dtype=float32), -0.17336872])
(40, [array([[ 0.28176975,  0.48700568,  0.09774131]], dtype=float32), -0.17469805])
(45, [array([[ 0.28306174,  0.48917308,  0.09823143]], dtype=float32), -0.17547914])
(50, [array([[ 0.28370172,  0.49024412,  0.09847598]], dtype=float32), -0.17586514])
